In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import array

from scipy.stats import norm
from scipy.optimize import brentq
from scipy.optimize import least_squares
from scipy.integrate import quad

import matplotlib.pylab as plt

import warnings
warnings.filterwarnings("ignore")

## For SPX

## Black Scholes Model

In [2]:
#Given,On 1-Dec-2020, the S&P500 (SPX) index value was 3662.45, while the SPDR S&P500 Exchange Traded Fund (SPY) stock price was 366.02
S_spx = 3662.45
S_spy = 366.02

In [3]:
df = pd.read_csv('SPX_options.csv')
rates_df = pd.read_csv('zero_rates_20201201.csv')

In [4]:
#Filter for only expiries of 15th Jan 2021
df = df[df["exdate"] == df["exdate"].unique()[1]].reset_index(drop = True)

In [5]:
df["date"] = pd.to_datetime(df['date'], format='%Y%m%d')
df["exdate"] = pd.to_datetime(df['exdate'], format='%Y%m%d')
df['Strike'] = (df['strike_price'])/1000
df['Payoff'] = np.where(df['cp_flag']=="C","Call","Put")
df['Price'] = (df['best_bid'] + df['best_offer'])/2

#Below values are constant
T = (df["exdate"][0] - df["date"][0]).days/365
r = np.interp((df["exdate"][0] - df["date"][0]).days, rates_df["days"], rates_df["rate"])/100
F = S_spx*np.exp(r*T)

In [6]:
#Define the Black Scholes implied vol calculator function 

def BlackScholesVanillaCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesVanillaPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

def impliedVolatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesVanillaCall(S, K, r, x, T),
                                a = 1e-12,b = 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesVanillaPut(S, K, r, x, T),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

In [7]:
df_atm = df[df['Strike'] == 3660].reset_index(drop=True)

In [8]:
#Calculate Market Implied Vols
df_atm["BS_ImpliedVol"] = False

for i in range(0,len(df_atm)):
    df_atm["BS_ImpliedVol"][i] = impliedVolatility(S_spx, df_atm['Strike'][i], r, df_atm['Price'][i], T, df_atm['Payoff'][i])

In [9]:
df_atm

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,Strike,Payoff,Price,BS_ImpliedVol
0,2020-12-01,2021-01-15,C,3660000,94.5,95.4,E,3660.0,Call,94.95,0.044298
1,2020-12-01,2021-01-15,P,3660000,94.8,95.5,E,3660.0,Put,95.15,0.270597


In [10]:
sigma_atm = (df_atm["BS_ImpliedVol"][0] + df_atm["BS_ImpliedVol"][1]) / 2
sigma_atm

0.15744743065886121

### Payoff function:

For any twice-differentiable payoff $h(S_T)$, Breeden-Litzenberger states that

\begin{equation*}
    \begin{split}
      V_0 = e^{-rT}h(F) + \underbrace{\int_0^{F}h''(K)P(K)\;dK}_{\mbox{put integral}} + \underbrace{\int_{F}^{\infty}h''(K)C(K)\;dK}_{\mbox{call integral}}\\
    \end{split}
\end{equation*}

  

In [11]:
def h(St):
    return St**(1/3) + 1.5* math.log(St) + 10

def hprimeprime(K):
    return -2/(9*K**(5/3)) - 1.5/K**(2)

In [12]:
def BlackScholesCallIntegrand(K, S, r, sigma, T):
    price = BlackScholesVanillaCall(S, K, r, sigma, T) * hprimeprime(K)
    return price

def BlackScholesPutIntegrand(K, S, r, sigma, T):
    price = BlackScholesVanillaPut(S, K, r, sigma, T) * hprimeprime(K)
    return price

In [13]:
I_put = quad(lambda x: BlackScholesPutIntegrand(x, S_spx, r, sigma_atm, T), 0, F)
I_call = quad(lambda x: BlackScholesCallIntegrand(x, S_spx, r, sigma_atm, T), F, 1000000)

In [14]:
Vo = np.exp(-r*T) * h(F) + I_put[0] + I_call[0]
print(Vo)

36.938003054940694


### Model-free integrated variance:

Model-free integrated variance can be calculated from the below formula:

\begin{equation*}
\begin{split}
    \mathbb{E}\left[\int_0^T\sigma_t^2 \;dt\right] = 2e^{rT} \left(\int_0^{F}\frac{P(K)}{K^2}\;dK + \int_{F}^{\infty}\frac{C(K)}{K^2}\;dK\right)
    \end{split}
\end{equation*}


In [15]:
def BlackScholesCallIntegrand(K, S, r, sigma, T):
    price = BlackScholesVanillaCall(S, K, r, sigma, T) / K**2
    return price

def BlackScholesPutIntegrand(K, S, r, sigma, T):
    price = BlackScholesVanillaPut(S, K, r, sigma, T) / K**2
    return price

In [16]:
I_put = quad(lambda x: BlackScholesPutIntegrand(x, S_spx, r, sigma_atm, T), 0, F)
I_call = quad(lambda x: BlackScholesCallIntegrand(x, S_spx, r, sigma_atm, T), F, 1000000)

In [17]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print(E_var)

0.003056263572461528


In [18]:
sigma_atm*sigma_atm*T

0.003056263572461537

## Bachelier Model

### Payoff function:

In [19]:
def BachelierCall(S, K, r, sigma, T):
    d1 = (S-K)/(sigma*np.sqrt(T))
    return ((S-K)*norm.cdf(d1) + sigma*np.sqrt(T)*norm.pdf(d1)) * np.exp(-r*T)

def BachelierPut(S, K, r, sigma, T):
    d1 = (S-K)/(sigma*np.sqrt(T))
    return ((K-S)*norm.cdf(-d1) + sigma*np.sqrt(T)*norm.pdf(-d1)) * np.exp(-r*T)

def BachelierCallIntegrand(K, S, r, sigma, T):
    price = BachelierCall(S, K, r, sigma, T) * hprimeprime(K)
    return price

def BachelierPutIntegrand(K, S, r, sigma, T):
    price = BachelierPut(S, K, r, sigma, T) * hprimeprime(K)
    return price

In [20]:
I_put = quad(lambda x: BachelierPutIntegrand(x, S_spx, r, sigma_atm, T), 0, F)
I_call = quad(lambda x: BachelierCallIntegrand(x, S_spx, r, sigma_atm, T), F, 1000000)

In [21]:
Vo = np.exp(-r*T) * h(F) + I_put[0] + I_call[0]
print(Vo)

36.9438555624206


### Model-free integrated variance:

In [22]:
def BachelierCallIntegrand(K, S, r, sigma, T):
    price = BachelierCall(S, K, r, sigma, T) / K**2
    return price

def BachelierPutIntegrand(K, S, r, sigma, T):
    price = BachelierPut(S, K, r, sigma, T) / K**2
    return price

In [23]:
I_put = quad(lambda x: BachelierPutIntegrand(x, S_spx, r, sigma_atm, T), 0, F)
I_call = quad(lambda x: BachelierCallIntegrand(x, S_spx, r, sigma_atm, T), F, 1000000)

In [24]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print(E_var)

0.000634088352319479


## SABR Model

In [25]:
#Remove ITM Call and Puts
df["Keep"] = np.where((df['Strike']>F) & (df['Payoff'] == "Call"),True, False)
df["Keep"] = np.where((df['Strike']<F) & (df['Payoff'] == "Put"),True, df["Keep"])
df = df[df["Keep"]==True].reset_index(drop = True)

In [26]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [27]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesVanillaCall(S, K, r, sabr_vol, T)

def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesVanillaPut(S, K, r, sabr_vol, T)

From Previous Part, for SPX options of 15th Jan expiry,
Calibrated SABR model parameters: alpha = 1.999, beta = 0.7, rho = -0.633, nu = 2.664

In [28]:
alpha = 1.999
beta = 0.7
rho = -0.633
nu = 2.664

### Payoff function:

In [29]:
def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) * hprimeprime(K)
    return price

def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) * hprimeprime(K)
    return price

In [30]:
I_put = quad(lambda x: sabrputintegrand(x, S_spx, r, T, alpha, beta, rho, nu), 0, F)
I_call = quad(lambda x: sabrcallintegrand(x, S_spx, r, T, alpha, beta, rho, nu), F, 1000000)

In [31]:
Vo = np.exp(-r*T) * h(F) + I_put[0] + I_call[0]
print(Vo)

36.929430043886704


### Model-free integrated variance:

In [32]:
def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / (K**2)
    return price

def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / (K**2)
    return price

In [33]:
I_put = quad(lambda x: sabrputintegrand(x, S_spx, r, T, alpha, beta, rho, nu), 0, F)
I_call = quad(lambda x: sabrcallintegrand(x, S_spx, r, T, alpha, beta, rho, nu), F, 1000000)

In [34]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print(E_var)

0.006928123898552289


# For SPY

## Black Scholes Model

In [35]:
df = pd.read_csv('SPY_options.csv')
rates_df = pd.read_csv('zero_rates_20201201.csv')

In [36]:
#Filter for only expiries of 15th Jan 2021
df = df[df["exdate"] == df["exdate"].unique()[1]].reset_index(drop = True)

In [37]:
df["date"] = pd.to_datetime(df['date'], format='%Y%m%d')
df["exdate"] = pd.to_datetime(df['exdate'], format='%Y%m%d')
df['Strike'] = (df['strike_price'])/1000
df['Payoff'] = np.where(df['cp_flag']=="C","Call","Put")
df['Price'] = (df['best_bid'] + df['best_offer'])/2

#Below values are constant
T = (df["exdate"][0] - df["date"][0]).days/365
r = np.interp((df["exdate"][0] - df["date"][0]).days, rates_df["days"], rates_df["rate"])
F = S_spy*np.exp(r*T)

In [38]:
#Define the Black Scholes implied vol calculator function 
def american_call(S, K, r, sigma, T, steps):
    R = (1+r)**(T/steps)
    u = np.exp(sigma*np.sqrt(T/steps))
    d = 1.0/u
    p_up = (R-d)/(u-d)
    p_down = 1.0-p_up

    prices = array.array('d', (0 for i in range(0, steps+1)))

    prices[0] = S*pow(d, steps)
    uu = u*u
    for i in range(1, steps+1):
        prices[i] = uu*prices[i-1]

    call_values = array.array('d', (0 for i in range(0, steps+1)))
    for i in range(0, steps+1):
        call_values[i] = max(0.0, (prices[i]-K))

    for step in range(steps-1, -1, -1):
        for i in range(0, step+1):
            call_values[i] = (p_up*call_values[i+1]+p_down*call_values[i])/R
            prices[i] = d*prices[i+1]
            call_values[i] = max(call_values[i], 1.0*(prices[i]-K))

    return call_values[0]


def american_put(S, K, r, sigma, T, steps):
    R = (1+r)**(T/steps)
    u = np.exp(sigma*np.sqrt(T/steps))
    uu = u*u
    d = 1.0/u
    p_up = (R-d)/(u-d)
    p_down = 1.0-p_up
    prices = array.array('d', (0 for i in range(0, steps+1)))
    prices[0] = S*pow(d, steps)

    for i in range(1, steps+1):
        prices[i] = uu*prices[i-1]

    put_values = array.array('d', (0 for i in range(0, steps+1)))

    for i in range(0, steps+1):
        put_values[i] = max(0.0, (K-prices[i]))

    for step in range(steps-1, -1, -1):
        for i in range(0, step+1):
            put_values[i] = (p_up*put_values[i+1]+p_down*put_values[i])/R
            prices[i] = d*prices[i+1]
            put_values[i] = max(put_values[i], (K-prices[i]))
    return put_values[0]

def impliedVolatility_americanoptions(S, K, r, price, T, payoff, steps):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                american_call(S, K, r, x, T, steps),
                                a = 1e-12,b = 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                american_put(S, K, r, x, T, steps),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

In [39]:
df_atm = df[df['Strike'] == 366].reset_index(drop=True)

In [40]:
#Calculate Market Implied Vols
df_atm["BS_ImpliedVol"] = False

for i in range(0,len(df_atm)):
    df_atm["BS_ImpliedVol"][i] = impliedVolatility_americanoptions(S_spy, df_atm['Strike'][i], r, df_atm['Price'][i], T, df_atm['Payoff'][i], 2000)

In [41]:
sigma_atm = (df_atm["BS_ImpliedVol"][0] + df_atm["BS_ImpliedVol"][1]) / 2
sigma_atm

0.12839541739221888

### Payoff function:

For any twice-differentiable payoff $h(S_T)$, Breeden-Litzenberger states that

\begin{equation*}
    \begin{split}
      V_0 = e^{-rT}h(F) + \underbrace{\int_0^{F}h''(K)P(K)\;dK}_{\mbox{put integral}} + \underbrace{\int_{F}^{\infty}h''(K)C(K)\;dK}_{\mbox{call integral}}\\
    \end{split}
\end{equation*}

  

In [42]:
def h(St):
    return St**(1/3) + 1.5* math.log(St) + 10

def hprimeprime(K):
    return -2/(9*K**(5/3)) - 1.5/K**(2)

In [43]:
def BlackScholesCallIntegrand(K, S, r, sigma, T):
    price = BlackScholesVanillaCall(S, K, r, sigma, T) * hprimeprime(K)
    return price

def BlackScholesPutIntegrand(K, S, r, sigma, T):
    price = BlackScholesVanillaPut(S, K, r, sigma, T) * hprimeprime(K)
    return price

In [44]:
I_put = quad(lambda x: BlackScholesPutIntegrand(x, S_spy, r, sigma_atm, T), 0, F)
I_call = quad(lambda x: BlackScholesCallIntegrand(x, S_spy, r, sigma_atm, T), F, 5000)

In [45]:
Vo = np.exp(-r*T) * h(F) + I_put[0] + I_call[0]
print(Vo)

25.45079351880009


### Model-free integrated variance:

Model-free integrated variance can be calculated from the below formula:

\begin{equation*}
\begin{split}
    \mathbb{E}\left[\int_0^T\sigma_t^2 \;dt\right] = 2e^{rT} \left(\int_0^{F}\frac{P(K)}{K^2}\;dK + \int_{F}^{\infty}\frac{C(K)}{K^2}\;dK\right)
    \end{split}
\end{equation*}


In [46]:
def BlackScholesCallIntegrand(K, S, r, sigma, T):
    price = BlackScholesVanillaCall(S, K, r, sigma, T) / K**2
    return price

def BlackScholesPutIntegrand(K, S, r, sigma, T):
    price = BlackScholesVanillaPut(S, K, r, sigma, T) / K**2
    return price

In [47]:
I_put = quad(lambda x: BlackScholesPutIntegrand(x, S_spy, r, sigma_atm, T), 0, F)
I_call = quad(lambda x: BlackScholesCallIntegrand(x, S_spy, r, sigma_atm, T), F, 5000)

In [48]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print(E_var)

0.0020324445050123297


In [49]:
sigma_atm*sigma_atm*T

0.002032444505012314

## Bachelier Model


### Payoff function:

In [50]:
def BachelierCall(S, K, r, sigma, T):
    d1 = (S-K)/(sigma*np.sqrt(T))
    return ((S-K)*norm.cdf(d1) + sigma*np.sqrt(T)*norm.pdf(d1)) * np.exp(-r*T)

def BachelierPut(S, K, r, sigma, T):
    d1 = (S-K)/(sigma*np.sqrt(T))
    return ((K-S)*norm.cdf(-d1) + sigma*np.sqrt(T)*norm.pdf(-d1)) * np.exp(-r*T)

def BachelierCallIntegrand(K, S, r, sigma, T):
    price = BachelierCall(S, K, r, sigma, T) * hprimeprime(K)
    return price

def BachelierPutIntegrand(K, S, r, sigma, T):
    price = BachelierPut(S, K, r, sigma, T) * hprimeprime(K)
    return price

In [51]:
I_put = quad(lambda x: BachelierPutIntegrand(x, S_spy, r, sigma_atm, T), 0, F)
I_call = quad(lambda x: BachelierCallIntegrand(x, S_spy, r, sigma_atm, T), F, 5000)

In [52]:
Vo = np.exp(-r*T) * h(F) + I_put[0] + I_call[0]
print(Vo)

25.45291012574156


### Model-free integrated variance:

In [53]:
def BachelierCallIntegrand(K, S, r, sigma, T):
    price = BachelierCall(S, K, r, sigma, T) / K**2
    return price

def BachelierPutIntegrand(K, S, r, sigma, T):
    price = BachelierPut(S, K, r, sigma, T) / K**2
    return price

In [54]:
I_put = quad(lambda x: BachelierPutIntegrand(x, S_spy, r, sigma_atm, T), 0, F)
I_call = quad(lambda x: BachelierCallIntegrand(x, S_spy, r, sigma_atm, T), F, 5000)

In [55]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print(E_var)

0.0006341032954980808


In [56]:
sigma_atm*sigma_atm*T

0.002032444505012314

## SABR Model

In [57]:
#Remove ITM Call and Puts
df["Keep"] = np.where((df['Strike']>F) & (df['Payoff'] == "Call"),True, False)
df["Keep"] = np.where((df['Strike']<F) & (df['Payoff'] == "Put"),True, df["Keep"])
df = df[df["Keep"]==True].reset_index(drop = True)

In [58]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [59]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesVanillaCall(S, K, r, sabr_vol, T)

def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesVanillaPut(S, K, r, sabr_vol, T)

From Previous Part, for SPY options of 15th Jan expiry,
Calibrated SABR model parameters: alpha = 0.937, beta = 0.7, rho = -0.577, nu = 2.731

In [60]:
alpha = 0.937
beta = 0.7
rho = -0.577
nu = 2.731

### Payoff function:

In [61]:
def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) * hprimeprime(K)
    return price

def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) * hprimeprime(K)
    return price

In [62]:
I_put = quad(lambda x: sabrputintegrand(x, S_spy, r, T, alpha, beta, rho, nu), 0, F)
I_call = quad(lambda x: sabrcallintegrand(x, S_spy, r, T, alpha, beta, rho, nu), F, 5000)

In [63]:
Vo = np.exp(-r*T) * h(F) + I_put[0] + I_call[0]
print(Vo)

25.444606795353593


### Model-free integrated variance:

In [64]:
def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / (K**2)
    return price

def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / (K**2)
    return price

In [65]:
I_put = quad(lambda x: sabrputintegrand(x, S_spy, r, T, alpha, beta, rho, nu), 0, F)
I_call = quad(lambda x: sabrcallintegrand(x, S_spy, r, T, alpha, beta, rho, nu), F, 5000)

In [66]:
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print(E_var)

0.006348801100944951
